In [193]:
import pandas as pd 
import numpy as np 

In [194]:
def generate_time_series(sample_size, n_steps): 
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, sample_size, 1) 
    time = np.linspace(0, 1, n_steps) 
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) #wave1+ 
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) #wave2+ 
    series += 0.1 * (np.random.rand(sample_size, n_steps) - 0.5)  #noise 
    return series[..., np.newaxis].astype(np.float32)

# A) Create a dataset of 10,000 samples with 51 time steps each (Note: the 51st time step should be used as the label) 

In [195]:
df=generate_time_series(10000,51)

In [211]:
x=df[:,:-1,:]

In [212]:
x.shape

(10000, 50, 1)

In [213]:
y=df[:,-1,:]

In [214]:
y.shape

(10000, 1)

# B) Split the dataset in the ratio training: validation: testing = 70:20:10. 

In [215]:
from sklearn.model_selection import train_test_split

In [216]:
x_train,X_test,y_train,Y_test = train_test_split(x,y, test_size=0.3, shuffle=False)

In [217]:
x_test,x_val,y_test,y_val = train_test_split(X_test,Y_test, test_size=0.33333, shuffle=False)

In [218]:
print("Training set shapes:", x_train.shape, y_train.shape)
print("Validation set shapes:", x_val.shape, y_val.shape)
print("Testing set shapes:", x_test.shape, y_test.shape)

Training set shapes: (7000, 50, 1) (7000, 1)
Validation set shapes: (1000, 50, 1) (1000, 1)
Testing set shapes: (2000, 50, 1) (2000, 1)


# C) Design, train, test and compare the performances of the following on the prediction of the value of  51st time step in the generated time series.  . 

## a. Fully connected neural network.

In [219]:
import tensorflow as tf 

In [205]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Flatten,Dense

In [207]:
ann=Sequential([
    Flatten(input_shape=[50,1]),
    Dense(1000,activation='relu'),
    Dense(500,activation='relu'),
    Dense(100,activation='relu'),
    Dense(1,activation='relu') # output layer
])

In the context of training a neural network for regression tasks (predicting a continuous value), using Mean Squared Error (MSE) as the loss function and Mean Absolute Error (MAE) as the evaluation metric is common practice. Here's why:

In [208]:
ann.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [210]:
x_train.shape

(7000, 50, 1)

In [152]:
ans = ann.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=32)

Epoch 1/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0649 - val_accuracy: 0.0000e+00 - val_loss: 0.0620
Epoch 2/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0677 - val_accuracy: 0.0000e+00 - val_loss: 0.0616
Epoch 3/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0671 - val_accuracy: 0.0000e+00 - val_loss: 0.0617
Epoch 4/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0658 - val_accuracy: 0.0000e+00 - val_loss: 0.0619
Epoch 5/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0678 - val_accuracy: 0.0000e+00 - val_loss: 0.0616
Epoch 6/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0000e+00 - loss: 0.0648 - val_accuracy: 0.0000e+00 - val_loss: 0.0617
Epoch 7/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0659 - val_accuracy: 0.0000e+00 - val_loss: 0.0616
Epoch 8/10
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/s

In [148]:
test_loss, test_mae = ann.evaluate(x_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test MAE: {test_mae}")


Test loss: 0.06616664677858353, Test MAE: 0.17127029597759247


## b. Simple RNN with one layer (output layer) 

In [181]:
from tensorflow.keras.layers import SimpleRNN

In [182]:
rnn1 = Sequential([
SimpleRNN(20,input_shape=[50, 1]),
    Dense(1)
])

C:\Users\godof\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [183]:
rnn1.compile(optimizer='adam',loss='mse',metrics=['mae'])

In [184]:
rnn1.fit(x_train,y_train,epochs=20,validation_data=(x_val,y_val))

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0793 - mae: 0.2161 - val_loss: 0.0130 - val_mae: 0.0913
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0104 - mae: 0.0816 - val_loss: 0.0071 - val_mae: 0.0674
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0065 - mae: 0.0645 - val_loss: 0.0053 - val_mae: 0.0585
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0051 - mae: 0.0572 - val_loss: 0.0045 - val_mae: 0.0535
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0043 - mae: 0.0527 - val_loss: 0.0042 - val_mae: 0.0516
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0040 - mae: 0.0504 - val_loss: 0.0036 - val_mae: 0.0480
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0037 - mae: 0.0491 - val_loss: 0.0036 - val_mae: 0.0478
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0035 - mae: 0.0475 - val_loss: 0.0034 - val_mae: 0.0464
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - lo

In [185]:
test_loss,test_mae = rnn1.evaluate(x_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test MAE: {test_mae}")

Test loss: 0.0033538108691573143, Test MAE: 0.046207327395677567


## e. Simple RNN with two hidden layers and one output layer. 

### c. Simple RNN with one hidden layer and one output layer. (same as previous)
## e. Simple RNN with two hidden layers and one output layer. (IMP)


In [188]:
#
rnn2 = Sequential([
SimpleRNN(20,input_shape=[50, 1],return_sequences=True),
SimpleRNN(20,input_shape=[50, 1],return_sequences=False),
    Dense(1)
])

In [189]:
rnn2.compile(optimizer='adam',loss='mse',metrics=['mae'])

In [190]:
rnn2.fit(x_train,y_train,epochs=20,validation_data=(x_val,y_val))

Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0868 - mae: 0.1866 - val_loss: 0.0049 - val_mae: 0.0555
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0045 - mae: 0.0536 - val_loss: 0.0039 - val_mae: 0.0500
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0035 - mae: 0.0476 - val_loss: 0.0035 - val_mae: 0.0471
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0034 - mae: 0.0471 - val_loss: 0.0036 - val_mae: 0.0481
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0032 - mae: 0.0457 - val_loss: 0.0033 - val_mae: 0.0464
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0031 - mae: 0.0452 - val_loss: 0.0030 - val_mae: 0.0439
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0031 - mae: 0.0447 - val_loss: 0.0033 - val_mae: 0.0468
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.0031 - mae: 0.0446 - val_loss: 0.0032 - val_mae: 0.0460
Epoch 9/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - l

In [192]:
test_loss,test_mae = rnn2.evaluate(x_test, y_test, verbose=0)
print(f"Test loss: {test_loss}, Test MAE: {test_mae}")

Test loss: 0.0029134443029761314, Test MAE: 0.04349382221698761
